In [58]:
import gzip
import numpy as np
import pandas as pd
from time import time

from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from time import perf_counter
import os
sns.set()

### Keras is the high-level API of TensorFlow ###

- Keras is functional in declarative program that tell what but not how in higher order.
- In symbolic language there is a clear separation between defining computation graph and compiling it which is mroe efficient for reusability.
- Statis language means of getting defined then run. We need to write the whole program before running.

### Using TensorFlow backend ###

In [35]:
root = './MNIST/raw/'
if not os.path.exists(root):
    os.mkdir(root)

## Prepare data ##

In [37]:
def read_mnist(images_path: str, labels_path: str):
    with gzip.open(labels_path, 'rb') as labelsFile:
        labels = np.frombuffer(labelsFile.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path,'rb') as imagesFile:
        length = len(labels)
        # Load flat 28x28 px images (784 px), and convert them to 28x28 px
        features = np.frombuffer(imagesFile.read(), dtype=np.uint8, offset=16) \
                        .reshape(length, 784) \
                        .reshape(length, 28, 28, 1)
        
    return features, labels

In [67]:
train = {}
test = {}

train['features'], train['labels'] = read_mnist(root+'train-images-idx3-ubyte.gz', root+'train-labels-idx1-ubyte.gz')
test['features'], test['labels'] = read_mnist(root+'t10k-images-idx3-ubyte.gz', root+'t10k-labels-idx1-ubyte.gz')

In [69]:
X_train, y_train = train['features'], to_categorical(train['labels'])
X_test, y_test = test['features'], to_categorical(test['labels'])

In [70]:
X_train,X_test=X_train/255.0, X_test/255.0
X_train,X_test = np.expand_dims(X_train, axis=-1), np.expand_dims(X_test, axis=-1)

 The model has a similar architecture that we built using PyTorch.

## Build the Model using Tensorflow ##

In [71]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(32, (5, 5), activation=tf.nn.relu)(inputs)
x =  tf.keras.layers.MaxPool2D((2, 2))(x)
x = tf.keras.layers.Conv2D(32, (5, 5), activation=tf.nn.relu)(x)
x =  tf.keras.layers.MaxPool2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu)(x)
x =  tf.keras.layers.MaxPool2D((2, 2))(x)
x =  tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(120, activation=tf.nn.relu)(x)
x = tf.keras.layers.Dense(84, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 8, 8, 32)          25632     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 1, 1, 64)          0   

## Define the Loss function and Optimizer ##

In [72]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

In [73]:
#Compile the model using sparse_categorical_crossentropy as we have 10 classes that we want to predict.
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [74]:
EPOCHS = 10
BATCH_SIZE = 128

## Train the Model ## 

In [76]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("mnist.h5", monitor='accuracy',verbose=1, save_best_only=True, save_weights_only=False,mode='auto', save_freq='epoch')
t1_start= perf_counter()
model.fit(X_train, y_train, epochs=5, verbose=2, 
                callbacks=[checkpoint])
t1_end= perf_counter()
print(" Time for training using Keras %f in(sec)" % (t1_end-t1_start))

Epoch 1/2
1875/1875 - 17s - loss: 0.0889 - accuracy: 0.9731

Epoch 00001: accuracy improved from -inf to 0.97310, saving model to mnist.h5
Epoch 2/2
1875/1875 - 17s - loss: 0.0512 - accuracy: 0.9843

Epoch 00002: accuracy improved from 0.97310 to 0.98433, saving model to mnist.h5
 Time for training using Keras 33.834193 in(sec)


 Notice that the avg_cost values decrease with each logged iteration. This means that the gradient descent algorithm is minimizing the cost function.

## Evaluate the Model ##

In [78]:
t1= perf_counter()
test_loss, test_acc = model.evaluate(X_test, y_test)
t2=perf_counter()
print("Eval accuracy using Keras is %.2f and execution time %.2f seconds" %((test_acc*100), (t2-t1)))

313/313 [==============================] - 1s 2ms/step - loss: 0.0389 - accuracy: 0.9880
Eval accuracy using Keras is 98.80 and execution time 0.87 seconds
